In [1]:
""" This script will do the following: 
        Resolving ordinals to function names when they appear,
        Converting both DLL names and function names to all lowercase,
        Removing the file extensions from imported module names,
        Building and storing the lowercased string in an ordered list,
        Generating the MD5 hash of the ordered list """

import pefile
import hashlib

def resolve_ordinals(pe):
    """
    Resolve ordinals to function names.
    """
    for entry in pe.DIRECTORY_ENTRY_IMPORT:
        module_name = entry.dll.decode('utf-8').lower().split('.')[0]  # Convert to lowercase and remove extension
        for imp in entry.imports:
            if imp.name is None:
                imp.name = f"ord_{imp.ordinal}"  # Resolve ordinals
        entry.dll = module_name.encode('utf-8')

def extract_imports(file_path):
    """
    Extract imports from a binary file (.exe).
    """
    pe = pefile.PE(file_path)
    resolve_ordinals(pe)
    
    imports = []
    for entry in pe.DIRECTORY_ENTRY_IMPORT:
        module_name = entry.dll.decode('utf-8').lower()
        functions = [imp.name.decode('utf-8').lower() for imp in entry.imports]
        imports.append((module_name, functions))
    
    return imports

def generate_md5(imports):
    """
    Generate MD5 hash of the ordered list of lowercased DLL names and function names.
    """
    ordered_list = sorted([f"{module_name}|{function}" for module_name, functions in imports for function in functions])
    md5 = hashlib.md5('\n'.join(ordered_list).encode()).hexdigest()
    return md5

def main(file_path):
    """
    Main function to execute the script.
    """
    imports = extract_imports(file_path)
    md5_hash = generate_md5(imports)
    return md5_hash

file_path = "rufus-4.4p.exe"
md5_hash = main(file_path)
print(f"MD5 Hash: {md5_hash}")


MD5 Hash: 7bc5aefd7566b24dabbb87a067f9c952
